# 필요한 라이브러리 불러오기

In [1]:
import sys, os, glob, natsort
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import colors, cm
from matplotlib.offsetbox import TextArea, AnnotationBbox
from mpl_toolkits.axes_grid1 import make_axes_locatable

# 디렉토리 설정 및 데이터 불러오기

In [2]:
os.chdir('C:/Users/hjm01/Desktop/겨울방학/alfven')
csv_list = natsort.natsorted(glob.glob('*.csv'))

parameter = ['r_sun','new_alf','vp_x','v','A','T','n_p']

for i in csv_list:
    Data_helios = pd.read_csv(i,index_col='Time',parse_dates=True)[parameter]
    try:
        Data = pd.concat([Data,Data_helios])
    except NameError:
        Data = Data_helios
        print("Data 변수 할당")

Data 변수 할당


# 필요한 변수 정의 및 데이터 특징에 따라 분류

In [3]:
Data['ex_t'] = (1.5*10**8)*Data['r_sun']/Data['v']

k = 1.38*10**(-16);e = 4.8*10**(-10); m = 1.67*10**(-24) #g
Data['logarithm'] = np.log((3*np.sqrt(k*Data['T'])**3)/(2*e**3*np.sqrt(Data['n_p']*np.pi)))

up = 4*np.sqrt(np.pi)*e**4*Data['n_p']*Data['logarithm']
down = 3*np.sqrt(m)*np.sqrt(k*Data['T'])**3

Data['chandran_fre'] = up/down

Data['c_n'] = Data['ex_t']*Data['chandran_fre']

Data1 = Data[Data['v'] >= 600].dropna(axis=0)
Data2 = Data[(Data['v'] <= 450)&(abs(Data['new_alf']) >= 0.7)].dropna(axis=0)
Data3 = Data[(Data['v'] <= 450)&(abs(Data['new_alf']) < 0.7)].dropna(axis=0)

# 히스토그램 그리기

In [4]:
h1,xed,yed = np.histogram2d(Data1['c_n'],Data1['A'],bins=(np.logspace(-2,1,100),np.logspace(-1,1,100)))
h2,_,_ = np.histogram2d(Data2['c_n'],Data2['A'],bins=(np.logspace(-2,1,100),np.logspace(-1,1,100)))
h3,_,_ = np.histogram2d(Data3['c_n'],Data3['A'],bins=(np.logspace(-2,1,100),np.logspace(-1,1,100)))

# 각 특징에 따라서 정규화

In [6]:
for idx, limit in enumerate([h1,h2,h3]):
    limit = np.array(limit)
    limit[limit <= 10] = 0
    globals()['h{}'.format(idx+1)] = limit

# 가장 Count가 높은 값으로 정규화
new_h1,new_h2,new_h3 = h1/h1.max(),h2/h2.max(),h3/h3.max()

for i,hh in enumerate([new_h1,new_h2,new_h3]):
    hh[hh == 0] = np.nan
    globals()['new_h{}'.format(i+1)] = hh

#pcolormesh로 그리기 위해 데이터프레임 회전
new_h1 = list(zip(*new_h1));new_h2 = list(zip(*new_h2));new_h3 = list(zip(*new_h3))

# 그래프를 그려 트렌드 파악

In [ ]:
fig, ax = plt.subplots(3,1,sharex=True,sharey=True,figsize=(10,15))

